In [ ]:
# Cell 1: 导入所需的库
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def preprocess_image(image_path, output_path=None):
   # 1. 读取图片
   img = cv2.imread(str(image_path))
   
   # 2. 转换到YCrCb空间
   ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
   
   # 3. YCrCb空间的肤色范围
   lower = np.array([0, 135, 85], dtype=np.uint8)
   upper = np.array([255, 180, 135], dtype=np.uint8)
   skin_mask = cv2.inRange(ycrcb, lower, upper)
   
   # 4. 改善掩码质量
   kernel = np.ones((5,5), np.uint8)
   skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel)
   skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel)
   
   # 5. 创建黑色背景，提取增强后的手部
   result = np.zeros_like(img)
   hand = cv2.convertScaleAbs(img, alpha=1.5, beta=30)
   result[skin_mask > 0] = hand[skin_mask > 0]
   
   # 6. 转灰度并二值化
   gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
   _, binary = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
   
   if output_path:
       cv2.imwrite(str(output_path), binary)
   return binary

In [ ]:
!ls

In [ ]:
# Cell 3: 处理并显示
input_image = "2024-12-01_14-47.png"
output_image = "output.jpg"

processed_img = preprocess_image(input_image, output_image)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(cv2.imread(input_image), cv2.COLOR_BGR2RGB))
plt.title('Original image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(processed_img, cmap='gray')
plt.title('Processed picture')
plt.axis('off')
plt.show()